# Part I. ETL Pipeline for Pre-Processing the Files

## PLEASE RUN THE FOLLOWING CODE FOR PRE-PROCESSING THE FILES

#### Import Python packages 

In [28]:
# Import Python packages 
import pandas as pd
import cassandra
import re
import os
import glob
import numpy as np
import json
import csv
from prettytable import PrettyTable

#### Creating list of filepaths to process original event csv data files

Extract files from the filepath and create list of files with full path

In [29]:
# checking current working directory
print(os.getcwd())

# Get your current folder and subfolder event data
filepath = os.getcwd() + '/event_data'

# Create a for loop to create a list of files and collect each filepath
for root, dirs, files in os.walk(filepath):
    
# join the file path and roots with the subdirectories using glob
    file_path_list = glob.glob(os.path.join(root,'*'))
    #print(file_path_list)

/home/workspace


#### Processing the files to create the data file csv that will be used for Apache Casssandra tables

In [30]:
# initiating an empty list of rows that will be generated from each file
full_data_rows_list = [] 
    
# for every filepath in the file path list 
for f in file_path_list:

# reading csv file 
    with open(f, 'r', encoding = 'utf8', newline='') as csvfile: 
        # creating a csv reader object 
        csvreader = csv.reader(csvfile) 
        #using next method advances the file reader by a line, this is a way to skip header and read the records from second line
        next(csvreader)
        
 # extracting each data row one by one and append it        
        for line in csvreader:
            #print(line)
            full_data_rows_list.append(line) 
            
# uncomment the code below if you would like to get total number of rows 
print(len(full_data_rows_list))
# uncomment the code below if you would like to check to see what the list of event data rows will look like
#print(full_data_rows_list)

# creating a smaller event data csv file called event_datafile_full csv that will be used to insert data into the \
# Apache Cassandra tables
csv.register_dialect('myDialect', quoting=csv.QUOTE_ALL, skipinitialspace=True)

with open('event_datafile_new.csv', 'w', encoding = 'utf8', newline='') as f:
    writer = csv.writer(f, dialect='myDialect')
    writer.writerow(['artist','firstName','gender','itemInSession','lastName','length',\
                'level','location','sessionId','song','userId'])
    for row in full_data_rows_list:
        if (row[0] == ''):
            continue
        writer.writerow((row[0], row[2], row[3], row[4], row[5], row[6], row[7], row[8], row[12], row[13], row[16]))


8056


In [31]:
# check the number of rows in your csv file
with open('event_datafile_new.csv', 'r', encoding = 'utf8') as f:
    print(sum(1 for line in f))

6821


# Part II. Complete the Apache Cassandra coding portion of your project. 

## Now you are ready to work with the CSV file titled <font color=red>event_datafile_new.csv</font>, located within the Workspace directory.  The event_datafile_new.csv contains the following columns: 
- artist 
- firstName of user
- gender of user
- item number in session
- last name of user
- length of the song
- level (paid or free song)
- location of the user
- sessionId
- song title
- userId

The image below is a screenshot of what the denormalized data should appear like in the <font color=red>**event_datafile_new.csv**</font> after the code above is run:<br>

<img src="images/image_event_datafile_new.jpg">

## Begin writing your Apache Cassandra code in the cells below

#### Creating a Cluster

In [32]:
# This should make a connection to a Cassandra instance your local machine 
# (127.0.0.1)

from cassandra.cluster import Cluster
try:
    cluster = Cluster(['127.0.0.1'])
    # To establish connection and begin executing queries, need a session
    session = cluster.connect()
except Exception as e:
    print(e)

#### Create Keyspace

In [33]:
# Create a Keyspace 
try:
    session.execute("""
    CREATE KEYSPACE IF NOT EXISTS udacity
    WITH REPLICATION = 
    {'class': 'SimpleStrategy', 'replication_factor':1}
    """)
except Exception as e:
    print(e)

#### Set Keyspace

In [34]:
# Set KEYSPACE to the keyspace specified above
try:
    session.set_keyspace('udacity')
except Exception as e:
    print(e)

### Now we need to create tables to run the following queries. Remember, with Apache Cassandra you model the database tables on the queries you want to run.

## Create queries to ask the following three questions of the data

### 1. Give me the artist, song title and song's length in the music app history that was heard during  sessionId = 338, and itemInSession  = 4


### 2. Give me only the following: name of artist, song (sorted by itemInSession) and user (first and last name) for userid = 10, sessionid = 182
    

### 3. Give me every user name (first and last) in my music app history who listened to the song 'All Hands Against His Own'




#### Create a table to show song played in a session at a specified position.
#### This table gives the result set of song in a specified position within a session.
#### For this the primary key is session_id, item_in_session. Session_id is the partition key and item_in_session is the clustering key which allows ordering of rows.

In [35]:
"""
Query 1:  Give me the artist, song title and song's length in the music app history that was heard during
sessionId = 338, and itemInSession = 4
 drop and create table
"""
query_d = "DROP TABLE IF EXISTS song_usage"
query = "CREATE TABLE IF NOT EXISTS song_usage "
query = query + "(session_id int, item_in_session int, artist_name text, song_title text, song_length float, user_id int, PRIMARY KEY (session_id, item_in_session))"
try:
    session.execute(query_d)
    session.execute(query)
except Exception as e:
    print(e)                 

In [36]:
# read the data file that was created above and insert into the table to answer query1
file = 'event_datafile_new.csv'

with open(file, encoding = 'utf8') as f:
    csvreader = csv.reader(f)
    next(csvreader) # skip header
    for line in csvreader:
    ## INSERT statements into the `query` variable
        query = "INSERT INTO song_usage ( session_id, item_in_session, artist_name, song_title, song_length, user_id)"
        query = query + " VALUES (%s, %s, %s, %s, %s, %s)"
        ## Assign which column element should be assigned for each column in the INSERT statement.
        ## For e.g., to INSERT artist_name and user first_name, you would change the code below to `line[0], line[1]`
        session.execute(query, (pd.to_numeric(line[8]), pd.to_numeric(line[3]), line[0], line[9], pd.to_numeric(line[5]), pd.to_numeric(line[10])))

#### Do a SELECT to verify that the data have been inserted into each table

In [37]:
""" SELECT statement to verify the data was entered into the table
 Query 1:  Give me the artist, song title and song's length in the music app history that was heard during \
 sessionId = 338, and itemInSession = 4
"""
query = "select artist_name, song_title, song_length from song_usage where session_id = 338 and item_in_session = 4"
try:
    rows = session.execute(query)
except Exception as e:
    print(e)

#Create a list and add rows to the list which are then added to a data frame for clean display
#rowlist = []
t = PrettyTable(['Artist Name', 'Song Title', 'Song Length'])

for row in rows:
    t.add_row([row.artist_name, row.song_title, row.song_length])
    #rowlist.append({'Artist Name':row.artist_name, 'Song title':row.song_title, 'Song Length':row.song_length})

print(t)
#df = pd.DataFrame(rowlist)
#df

+-------------+---------------------------------+--------------------+
| Artist Name |            Song Title           |    Song Length     |
+-------------+---------------------------------+--------------------+
|  Faithless  | Music Matters (Mark Knight Dub) | 495.30731201171875 |
+-------------+---------------------------------+--------------------+


### Create a table to check the songs played by user in a session and insert into the table

#### Create a table to show list of songs a user listened to in a session.
#### This table gives the result set of songs a specified user listened to in a given session.
#### For this the primary key is user_id and session_id and clustering column is item_in_session so it can be listed 
#### in the order it was played by the user.

In [38]:
""" Query 2: Give me only the following: name of artist, song (sorted by itemInSession) and user (first and last name)\
 for userid = 10, sessionid = 182
 drop and create table and insert records into the table from the csv file
"""
query_d = "DROP TABLE IF EXISTS user_song"
query = "CREATE TABLE IF NOT EXISTS user_song "
query = query + "(user_id int, session_id int, item_in_session int, artist_name text, song_title text, user_name text, \
                    PRIMARY KEY ((user_id, session_id), item_in_session))"
try:
    session.execute(query_d)
    session.execute(query)
except Exception as e:
    print(e)

file = 'event_datafile_new.csv'

with open(file, encoding = 'utf8') as f:
    csvreader = csv.reader(f)
    next(csvreader) # skip header
    for line in csvreader:
    ## INSERT statements into the `query` variable
        query = "INSERT INTO user_song (user_id, session_id, item_in_session, artist_name, song_title, user_name)"
        query = query + " VALUES (%s, %s, %s, %s, %s, %s)"
        ## Assign which column element should be assigned for each column in the INSERT statement.
        ## For e.g., to INSERT artist_name and user first_name, you would change the code below to `line[0], line[1]`
        session.execute(query, (pd.to_numeric(line[10]), pd.to_numeric(line[8]), pd.to_numeric(line[3]), line[0], line[9], line[1]+' '+line[4]))

In [39]:
"""
Give me only the following: name of artist, song (sorted by itemInSession) and user (first and last name)
 for userid = 10, sessionid = 182
"""
query = "select item_in_session, artist_name, song_title, user_name from user_song where user_id = 10 and session_id = 182 order by item_in_session"

try:
    rows = session.execute(query)
except Exception as e:
    print(e)

#Create a list and add rows to the list which are then added to a data frame for clean display
t = PrettyTable(['Item in Session', 'Artist', 'Song Title', 'User Name'])

#rowlist = []
for row in rows:
    #print (row.artist_name, row.song_title, row.user_name, row.item_in_session)
    #rowlist.append({"Artist Name":row.artist_name, "Song Title":row.song_title, "User Name": row.user_name, "Session":row.item_in_session})
    t.add_row([row.item_in_session, row.artist_name, row.song_title, row.user_name])
#df=pd.DataFrame(rowlist)
#df
print(t)

+-----------------+-------------------+------------------------------------------------------+-------------+
| Item in Session |       Artist      |                      Song Title                      |  User Name  |
+-----------------+-------------------+------------------------------------------------------+-------------+
|        0        |  Down To The Bone |                  Keep On Keepin' On                  | Sylvie Cruz |
|        1        |    Three Drives   |                     Greece 2000                      | Sylvie Cruz |
|        2        | Sebastien Tellier |                      Kilometer                       | Sylvie Cruz |
|        3        |   Lonnie Gordon   | Catch You Baby (Steve Pitron & Max Sanna Radio Edit) | Sylvie Cruz |
+-----------------+-------------------+------------------------------------------------------+-------------+


#### Create a table to check the song history of users.
#### This table gives the result set of the users that listened to a specified song.
#### For this the primary key song_tile and user_id would be sufficient. However, in adding session_id 
#### it can also check if the same user listened to a song in multiple sessions.

In [40]:
"""
Drop and create the table. Insert records into the table from the csv file
Query 3: Give me every user name (first and last) in my music app history who listened to the song 'All Hands Against His Own'
"""
query_d = "DROP TABLE IF EXISTS song_history"
query = "CREATE TABLE IF NOT EXISTS song_history "
query = query + "(song_title text, user_id int, session_id int, artist_name text, user_name text, item_in_session int, \
                    PRIMARY KEY (song_title, user_id, session_id))"
try:
    session.execute(query_d)
    session.execute(query)
except Exception as e:
    print(e)

file = 'event_datafile_new.csv'

with open(file, encoding = 'utf8') as f:
    csvreader = csv.reader(f)
    next(csvreader) # skip header
    for line in csvreader:
    ## INSERT statements into the `query` variable
        query = "INSERT INTO song_history (song_title, user_id, session_id, artist_name, user_name, item_in_session)"
        query = query + " VALUES (%s, %s, %s, %s, %s, %s)"
        ## Assign which column element should be assigned for each column in the INSERT statement.
        ## For e.g., to INSERT artist_name and user first_name, you would change the code below to `line[0], line[1]`
        session.execute(query, (line[9], pd.to_numeric(line[10]), pd.to_numeric(line[8]), line[0], line[1]+' '+line[4], pd.to_numeric(line[3])))
                    

In [27]:
"""
## Give me every user name (first and last) in my music app history who listened to the song 'All Hands Against His Own'
"""
query = "select user_name, song_title from song_history where song_title = 'All Hands Against His Own'"
try:
    rows = session.execute(query)
except Exception as e:
    print(e)
#Create a list and add rows to the list which are then added to a data frame for clean display
#rowlist = []
t = PrettyTable([ 'User Name', 'Song Title'])
for row in rows:
    #print (row.user_name, row.song_title)
    #rowlist.append({"User Name":row.user_name, "Song Title":row.song_title})
    t.add_row([row.user_name, row.song_title])
print(t)
#df=pd.DataFrame(rowlist)
#df

+------------------+---------------------------+
|    User Name     |         Song Title        |
+------------------+---------------------------+
| Jacqueline Lynch | All Hands Against His Own |
|   Tegan Levine   | All Hands Against His Own |
|   Sara Johnson   | All Hands Against His Own |
+------------------+---------------------------+


### Drop the tables before closing out the sessions

In [41]:
## Drop the table before closing out the sessions
query_d = "DROP TABLE IF EXISTS song_usage"
try:
    session.execute(query_d)
except Exception as e:
    print(e)

query_d = "DROP TABLE IF EXISTS user_song"
try:
    session.execute(query_d)
except Exception as e:
    print(e)

query_d = "DROP TABLE IF EXISTS song_history"
try:
    session.execute(query_d)
except Exception as e:
    print(e)


### Close the session and cluster connection¶

In [42]:
session.shutdown()
cluster.shutdown()